In [1]:
import pandas as pd
import numpy as np 
import janitor 
from io import StringIO
from janitor.utils import _not_equal_indices

In [2]:
data = """ A    B    C      D          E
    0  1  0.0   r    False    1970-01-01
    1  1  0.0   r    False    1970-01-01"""
    
df = pd.read_csv(StringIO(data), sep='\s{2,}', engine='python', parse_dates=['E'])

In [3]:
data = """Integers  Numeric  Floats   Strings  Booleans      Dates    Dates_Right
    0         0      1.0     0.0     r         False     1970-01-01  1970-01-01
    """

right = pd.read_csv(StringIO(data), sep='\s{2,}', engine='python', parse_dates=['Dates', 'Dates_Right'])

In [4]:
df

,A,B,C,D,E
0,1,0.0,r,False,1970-01-01
1,1,0.0,r,False,1970-01-01


In [5]:
right

,Integers,Numeric,Floats,Strings,Booleans,Dates,Dates_Right
0,0,1.0,0.0,r,False,1970-01-01,1970-01-01


In [6]:
_not_equal_indices(df.A, right.Integers)

(array([0, 1]), array([0, 0]))

In [7]:
right.Dates.sort_values()

0   1970-01-01
Name: Dates, dtype: datetime64[ns]

In [8]:
dict(zip(right.Dates, right.index))

{Timestamp('1970-01-01 00:00:00'): 0}

In [9]:
filters = ['A', 'Integers', 'B', 'Numeric', 'E', 'Dates']

actual = df.conditional_join(
        right,
        ('A', 'Integers', ">"),
        ('B', 'Numeric', "<"),
        ('E', 'Dates', '!='),
        how="inner",
        sort_by_appearance=True,
    )
actual 

# actual = actual.droplevel(level=0, axis=1)
# actual = actual.filter(filters)


In [10]:
expected = (
        df.assign(t=1)
        .merge(right.assign(t=1), on="t")
        .query("A > Integers and B < Numeric and E != Dates")
        .reset_index(drop=True)
    )

expected = expected.filter(filters)

expected

,A,Integers,B,Numeric,E,Dates
